In [5]:
import pandas as pd
import numpy as np

try:
    import gensim
except:
    !pip install gensim
    import gensim
    
try:
    import keras
except:
    !pip install tensorflow
    !pip install keras
    import keras
    
try:
    from sklearn.model_selection import train_test_split
except:
    !pip install sklearn
    from sklearn.model_selection import train_test_split

try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib
    import matplotlib.pyplot as plt
    
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [6]:
def file_reader(file_location):
    if(file_location.endswith('csv')):
        return pd.read_csv( file_location , engine = 'python', index_col=0)
    elif (file_location.endswith('tsv')):
        return pd.read_csv( file_location , engine = 'python' ,sep = '\t')

In [7]:
def read_dataset( file_location ):
    df = file_reader(file_location)
    token_list  = []
    for i in range( len( df ) ):
        token_list.append( df['TEXT'][i].split() )
    df['TOKENS'] = token_list
    return df

In [11]:
def tokens_to_sequence( tokenizer , texts , max_length ):
    sequences   = tokenizer.texts_to_sequences( texts )
    padded_data = pad_sequences( sequences , maxlen=max_length , padding='post' )
    return padded_data
    
def tokenize_dataset(df, max_length):
    num_words   = len(sorted(set([word for tok in df['TOKENS'] for word in tok])))
    tokenizer   = Tokenizer( num_words=num_words, lower=True, char_level=False , oov_token = "<OOV>")
    tokenizer.fit_on_texts( df['TEXT'].tolist() )
    return (tokenizer, num_words)

def get_embeddings(word_index):
    word2vec_path = 'input_files/GoogleNews-vectors-negative300.bin'
    word2vec      = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
    embeddings = np.zeros( (len(word_index)+1, embedding_dim))
    for word,index in word_index.items():
        if word in word2vec :
            embeddings[index,:] = word2vec[word] 
    del word2vec
    return embeddings

def embedded_values(data, embeddings):
    emb = []
    for value in data:
        t = np.array( np.mean([embeddings[w] for w in value if w < len(embeddings)] or [np.zeros(300)], axis=0))
        emb.append(t)
    return np.array(emb)

In [12]:
inp = '3_data_preprocessed/'
trait_names = ['cEXT' , 'cNEU' , 'cAGR' , 'cCON' , 'cOPN']
sklearn_models = [
    svm.SVC(kernel='rbf', gamma = 1.0 , C = 10),
    GaussianNB(priors=[0.5,0.5]),
    LogisticRegression(),
    RandomForestClassifier(max_depth=2, random_state=0)
]
max_length = 300
embedding_dim = 300

In [13]:
df_essay             = read_dataset( 'processed_datasets/essays.csv' )
tokenizer, num_words = tokenize_dataset(df_essay, max_length)
embeddings           = get_embeddings(tokenizer.word_index)
essay_sequences      = tokens_to_sequence(tokenizer, df_essay['TEXT'], max_length)
X_Train              = embedded_values(essay_sequences, embeddings)

In [14]:
df_tweets           = file_reader( 'processed_datasets/tweets_processed.csv')
pre_tweet_seq    = tokens_to_sequence(tokenizer, df_tweets['PRETEXT'], max_length )
post_tweet_seq   = tokens_to_sequence(tokenizer, df_tweets['POSTTEXT'], max_length )

In [18]:
for model in sklearn_models:
    model_name = type(model).__name__
    print(model_name)
    
    df_preds = pd.DataFrame()
    df_preds['PERSON'] = df_tweets.index
    for trait in trait_names:
        scores = model.fit(X_Train, df_essay[trait])
        print()
        df_preds['pre_'+trait]  = model.predict(pre_tweet_seq)
        df_preds['post_'+trait] = model.predict(pre_tweet_seq)
        
    df_preds.to_csv('result/preds_'+model_name+'.csv')

SVC
GaussianNB
LogisticRegression
RandomForestClassifier
